# Set up libraries

In [ ]:
import os
import pandas as pd
pd.options.mode.chained_assignment = None
from firecloud import api as fapi
from google.cloud import bigquery

# Set up billing project and data path variables

In [ ]:
BILLING_PROJECT_ID = os.environ['GOOGLE_PROJECT']
WORKSPACE_NAMESPACE = os.environ['WORKSPACE_NAMESPACE']
WORKSPACE_NAME = os.environ['WORKSPACE_NAME']
WORKSPACE_BUCKET = os.environ['WORKSPACE_BUCKET']

WORKSPACE_ATTRIBUTES = fapi.get_workspace(WORKSPACE_NAMESPACE, WORKSPACE_NAME).json().get('workspace',{}).get('attributes',{})

GS_RELEASE_PATH = 'gs://amp-pd-data/releases/2020_v2release_1218'
GS_CLINICAL_RELEASE_PATH = f'{GS_RELEASE_PATH}/clinical'

GS_WGS_RELEASE_PATH = 'gs://amp-pd-genomics/releases/2020_v2release_1218/wgs'
GS_WGS_RELEASE_PLINK_PATH = os.path.join(GS_WGS_RELEASE_PATH, 'plink')
GS_WGS_RELEASE_GATK_PATH = os.path.join(GS_WGS_RELEASE_PATH, 'gatk')

BQ_RELEASE_DATASET = 'amp-pd-research.2020_v2release_1218'


print(BILLING_PROJECT_ID)
print(GS_CLINICAL_RELEASE_PATH)
print(GS_WGS_RELEASE_PLINK_PATH)
print(GS_WGS_RELEASE_GATK_PATH)

# Useful functions

In [ ]:
def bq_query(query):
    """Return the contents of a query against BigQuery"""
    return pd.read_gbq(
        query,
        project_id=BILLING_PROJECT_ID,
        dialect='standard')

# Querying Clinical Data

In [ ]:
clinical_tables = f"""
SELECT 
project_id, dataset_id, table_id, row_count, size_bytes 
FROM `{BQ_RELEASE_DATASET}.__TABLES__`
"""
bq_query(clinical_tables)

## Clinical duplicates

In [ ]:
dups = f"""
SELECT 
*
FROM `{BQ_RELEASE_DATASET}.amp_pd_participant_wgs_duplicates`
"""
duplicates = bq_query(dups)

## Covariates

In [ ]:
covariates = f"""
SELECT 
participant_id, sex, age_at_baseline 
FROM `{BQ_RELEASE_DATASET}.Demographics`
"""
covs = bq_query(covariates)

In [ ]:
covs = covs.drop_duplicates()
covsp = covs
covsp['IID'] = covsp['participant_id']
covsp = covsp[['participant_id', 'IID', 'sex', 'age_at_baseline']]
covsp['sex'] = covsp['sex'].astype(str)
covsp.sex[(covsp.sex == "Male")] = 1
covsp.sex[(covsp.sex == "Female")] = 2
covsp.columns = ['FID','IID', 'sex', 'age_at_baseline']

## Phenotypes

In [ ]:
phenotype = f"""
SELECT 
* 
FROM `{BQ_RELEASE_DATASET}.amp_pd_case_control`
"""
pheno = bq_query(phenotype)

In [ ]:
phenop = pheno
phenop['IID'] = phenop['participant_id']
phenop = phenop[['participant_id', 'IID', 'case_control_other_latest']]
phenop.columns = ['FID','IID', 'PHENO']
phenop = phenop[(phenop.PHENO != 'Other')]
phenop.PHENO[(phenop.PHENO == "Case")] = 2
phenop.PHENO[(phenop.PHENO == "Control")] = 1

In [ ]:
covsp.to_csv('plink_test_covs.tab', index=False, sep='\t')
phenop.to_csv('plink_test_pheno.tab', index=False, sep='\t')

# Data processing

## Prepare PLINK

In [ ]:
!wget http://s3.amazonaws.com/plink1-assets/plink_linux_x86_64_20190304.zip -P ~/bin/data_temp/

In [ ]:
!unzip -o ~/bin/data_temp/plink_linux_x86_64_20190304.zip -d ~/bin/

In [ ]:
!~/bin/plink

## Download data

In [ ]:
!gsutil cp gs://fc-99ee693a-0e54-48ff-8e90-34a48151bea5/notebooks/PCA_filtered_europeans.txt .

In [ ]:
!for N in `seq 22` X Y; \
do \
gsutil -mu ib2021-parkinson cp gs://amp-pd-genomics/releases/2020_v2release_1218/wgs/plink/bfile/chr${N}.* .; \
done   

## Update sex

In [ ]:
!for N in `seq 22` X Y; \
do \
~/bin/plink \
--bfile chr${N} \
--update-sex plink_test_covs.tab \
--make-bed \
--out chr${N}_updated_sex && \
rm chr${N}.*; \
done

## Update phenotype

In [ ]:
!for N in `seq 22` X Y; \
do \
~/bin/plink \
--bfile chr${N}_updated_sex \
--make-pheno plink_test_pheno.tab 2 \
--keep plink_test_pheno.tab \
--make-bed \
--out chr${N}_updated_pheno && \
rm chr${N}_updated_sex.*; \
done

## Ancestry

In [ ]:
!for N in `seq 22` X Y; \
do \
~/bin/plink \
--bfile chr${N}_updated_pheno \
--keep PCA_filtered_europeans.txt \
--make-bed \
--out chr${N}_after_ancestry && \
rm chr${N}_updated_pheno.*; \
done

## Relatedness

In [ ]:
!for N in `seq 22` X Y; \
do \
~/bin/plink --bfile chr${N}_after_ancestry --geno 0.05 --maf 0.05 --indep-pairwise 50 5 0.5 --out chr${N}_pruning && \
~/bin/plink --bfile chr${N}_after_ancestry --extract chr${N}_pruning.prune.in --make-bed --out chr${N}_pruned_data && \
~/bin/plink --bfile chr${N}_pruned_data --het --out chr${N}_prunedHet; \
done

In [ ]:
!wget https://cnsgenomics.com/software/gcta/bin/gcta_1.93.2beta.zip -P ~/bin/data_temp

In [ ]:
!unzip -o ~/bin/data_temp/gcta_1.93.2beta.zip -d ~/bin/

In [ ]:
!~/bin/gcta_1.93.2beta/gcta64

In [ ]:
!ls chr*_pruned_data* | cut -d . -f 1 | sort -u > pruned_files_list

In [ ]:
!~/bin/gcta_1.93.2beta/gcta64 --mbfile pruned_files_list --make-grm --out GRM_matrix --autosome --maf 0.05

In [ ]:
!~/bin/gcta_1.93.2beta/gcta64 --grm-cutoff 0.125 --grm GRM_matrix --out GRM_matrix_0125 --make-grm

In [ ]:
!for N in `seq 22` X Y; \
do \
~/bin/plink \
--bfile chr${N}_after_ancestry \
--keep GRM_matrix_0125.grm.id \
--make-bed \
--out chr${N}_after_ancestry_pihat && \
rm chr${N}_after_ancestry.* chr${N}_pruning.* chr${N}_pruned_data.* chr${N}_prunedHet.*; \
done

In [ ]:
!rm GRM_matrix_0125.* GRM_matrix.* pruned_files_list

## Missingness per variant

In [ ]:
!for N in `seq 22` X Y; \
do \
~/bin/plink \
--bfile chr${N}_after_ancestry_pihat \
--make-bed \
--out chr${N}_after_ancestry_pihat_mind \
--geno 0.05 && \
rm chr${N}_after_ancestry_pihat.*; \
done

## Missingness by haplotype

In [ ]:
!for N in `seq 22` X Y; \
do \
~/bin/plink --bfile chr${N}_after_ancestry_pihat_mind_missing1 --test-mishap --out chr${N}_missing_hap && \
awk '{if ($8 <= 0.0001) print $9 }' chr${N}_missing_hap.missing.hap > chr${N}_missing_haps_1E4.txt && \
sed 's/|/\
/g' chr${N}_missing_haps_1E4.txt > chr${N}_missing_haps_1E4_final.txt && \
~/bin/plink \
--bfile chr${N}_after_ancestry_pihat_mind_missing1 \
--exclude chr${N}_missing_haps_1E4_final.txt \
--make-bed \
--out chr${N}_after_ancestry_pihat_mind_missing2 && \
rm chr${N}_after_ancestry_pihat_mind_missing1.* chr${N}_missing_hap.* chr${N}_missing_haps_1E4.txt chr${N}_missing_haps_1E4_final.txt; \
done

## HWE

In [ ]:
!for N in `seq 22` X Y; \
do \
~/bin/plink \
--bfile chr${N}_after_ancestry_pihat_mind_missing2 \
--filter-controls \
--hwe 1E-4 \
--write-snplist \
--out chr${N}_HWE_snps && \
~/bin/plink \
--bfile chr${N}_after_ancestry_pihat_mind_missing2 \
--extract chr${N}_HWE_snps.snplist \
--make-bed \
--out chr${N}_FILTERED && \
rm chr${N}_HWE_snps.* chr${N}_after_ancestry_pihat_mind_missing2.*; \
done

## Filter sepcific cohorts

In [ ]:
cohort_participiant_id = f"""
SELECT DISTINCT t1.participant_id 
FROM ((SELECT participant_id FROM `amp-pd-research.2020_v2release_1218.amp_pd_case_control` 
WHERE (diagnosis_latest = "No PD Nor Other Neurological Disorder") 
OR (diagnosis_latest = "Parkinson's Disease") 
OR (diagnosis_latest = "Idiopathic PD"))) 
t1 INNER JOIN ((SELECT participant_id 
FROM `amp-pd-research.2020_v2release_1218_genomics.wgs_samples` 
WHERE (CRAM IS NOT NULL))) t2 ON t1.participant_id = t2.participant_id 
INNER JOIN ((SELECT participant_id 
FROM `amp-pd-research.2020_v2release_1218.amp_pd_participants` 
WHERE (study = "LBD") 
OR (study = "PPMI") 
OR (study = "PDBP") 
OR (study = "HBS") 
OR (study = "BioFIND"))) 
t3 ON t2.participant_id = t3.participant_id
"""
cohort = bq_query(cohort_participiant_id)

In [ ]:
cohort.columns = ['FID']
cohort['IID'] = cohort['FID']
cohort.to_csv(os.environ['GENE_NAME'] + '/no_LBD_cohort.tsv', index=False, sep='\t')

## Download Annovar, rvtests and supplementary files

In [ ]:
!wget http://www.openbioinformatics.org/annovar/download/0wgxR2rIVP/annovar.latest.tar.gz

In [ ]:
!tar xvf annovar.latest.tar.gz

In [ ]:
!perl annovar/annotate_variation.pl -buildver hg38 -downdb -webfrom annovar refGene annovar/humandb/
!perl annovar/annotate_variation.pl -buildver hg38 -downdb cytoBand annovar/humandb/
!perl annovar/annotate_variation.pl -buildver hg38 -downdb -webfrom annovar ensGene annovar/humandb/
!perl annovar/annotate_variation.pl -buildver hg38 -downdb -webfrom annovar exac03 annovar/humandb/ 
!perl annovar/annotate_variation.pl -buildver hg38 -downdb -webfrom annovar avsnp150 annovar/humandb/ 
!perl annovar/annotate_variation.pl -buildver hg38 -downdb -webfrom annovar dbnsfp41c annovar/humandb/
!perl annovar/annotate_variation.pl -buildver hg38 -downdb -webfrom annovar gnomad211_genome annovar/humandb/
!perl annovar/annotate_variation.pl -buildver hg38 -downdb -webfrom annovar ljb26_all annovar/humandb/
!perl annovar/annotate_variation.pl -buildver hg38 -downdb -webfrom annovar clinvar_20210501 annovar/humandb/

In [ ]:
!wget -q https://github.com/zhanxw/rvtests/releases/download/v2.1.0/rvtests_linux64.tar.gz -P ~/bin/

In [ ]:
!mkdir ~/bin/rvtests
!tar xvf ~/bin/rvtests_linux64.tar.gz -C ~/bin/rvtests

In [ ]:
!~/bin/rvtests/executable/rvtest

In [ ]:
!wget http://hgdownload.cse.ucsc.edu/goldenPath/hg38/database/refFlat.txt.gz
!gunzip refFlat.txt.gz

In [ ]:
!gsutil cp gs://fc-99ee693a-0e54-48ff-8e90-34a48151bea5/notebooks/create_cov_file.R .

## Annotate list of genes

In [ ]:
!genes="GALC,14,87837820,87993665 "`\
      `"SORL1,11,121452314,121633763 "` \
      `"ARSA,22,50622754,50628173 "` \
      `"GRN,17,44345246,44353106 "` \
      `"CTCB,8,11842524,11869448 "` \
      `"SCARB2,4,76158737,76234536 "` \
      `"FUCA1,1,23845077,23868290 "` \
      `"CTSD,11,1752755,1764573 "` \
      `"GBA2,9,35736866,35749228" && \
for gene in $genes;\
do \
GENE_NAME=`echo $gene | cut -d ',' -f 1` && \
CHR_N=`echo $gene | cut -d ',' -f 2` && \
START=`echo $gene | cut -d ',' -f 3` && \
END=`echo $gene | cut -d ',' -f 4` && \
mkdir -p ${GENE_NAME}/ && \
cp no_LBD_cohort.tsv ${GENE_NAME}/no_LBD_cohort.tsv && \
~/bin/plink \
--bfile chr${CHR_N}_FILTERED \
--chr $CHR_N \
--from-bp $START \
--keep ${GENE_NAME}/no_LBD_cohort.tsv \
--to-bp $END \
--make-bed \
--out ${GENE_NAME}/chr${CHR_N}_${GENE_NAME} && \
~/bin/plink \
--bfile ${GENE_NAME}/chr${CHR_N}_${GENE_NAME} \
--recode vcf-fid bgz \
--out ${GENE_NAME}/chr${CHR_N}_${GENE_NAME}_recode && \
tabix -f -p vcf ${GENE_NAME}/chr${CHR_N}_${GENE_NAME}_recode.vcf.gz && \
perl annovar/convert2annovar.pl \
--format vcf4 \
--allsample \
--withfreq ${GENE_NAME}/chr${CHR_N}_${GENE_NAME}_recode.vcf.gz \
--outfile ${GENE_NAME}/chr${CHR_N}_${GENE_NAME}_recode_convert && \
perl annovar/table_annovar.pl \
${GENE_NAME}/chr${CHR_N}_${GENE_NAME}_recode_convert \
annovar/humandb/ \
--buildver hg38 \
--thread `nproc` \
--out ${GENE_NAME}/chr${CHR_N}_${GENE_NAME}_recode_convert.annovar \
--remove \
--protocol refGene,cytoBand,ensGene,exac03,avsnp150,dbnsfp41c,ljb26_all,gnomad211_genome,clinvar_20210501 \
--operation g,r,g,f,f,f,f,f,f \
--nastring . && \
head -1 ${GENE_NAME}/chr${CHR_N}_${GENE_NAME}_recode_convert.annovar.hg38_multianno.txt \
> ${GENE_NAME}/chr${CHR_N}_${GENE_NAME}_header.txt && \
colct="$(wc -w ${GENE_NAME}/chr${CHR_N}_${GENE_NAME}_header.txt | cut -f1 -d ' ')" && \
cut -f1-$colct ${GENE_NAME}/chr${CHR_N}_${GENE_NAME}_recode_convert.annovar.hg38_multianno.txt \
> ${GENE_NAME}/chr${CHR_N}_${GENE_NAME}_recode_convert.annovar.trimmed.txt && \
Rscript --no-save create_cov_file.R ${GENE_NAME}/chr${CHR_N}_${GENE_NAME}.fam \
${GENE_NAME}/chr${CHR_N}_${GENE_NAME}_covariateFile.txt && \
~/bin/rvtests/executable/rvtest \
--noweb \
--inVcf ${GENE_NAME}/chr${CHR_N}_${GENE_NAME}_recode.vcf.gz \
--kernel skat,skato \
--geneFile refFlat.txt \
--pheno ${GENE_NAME}/chr${CHR_N}_${GENE_NAME}_covariateFile.txt \
--pheno-name PHENO \
--covar ${GENE_NAME}/chr${CHR_N}_${GENE_NAME}_covariateFile.txt \
--covar-name SEX,AGE \
--out ${GENE_NAME}/chr${CHR_N}_${GENE_NAME}.all_rare \
--numThread `nproc` \
--freqUpper 0.01 && \
awk '$9 ~ /nonsyn/ {print $1" "$2" "$2" "$7}' ${GENE_NAME}/chr${CHR_N}_${GENE_NAME}_recode_convert.annovar.trimmed.txt \
> ${GENE_NAME}/chr${CHR_N}_${GENE_NAME}.all_coding_NS.txt && \
~/bin/plink \
--bfile ${GENE_NAME}/chr${CHR_N}_${GENE_NAME} \
--extract range ${GENE_NAME}/chr${CHR_N}_${GENE_NAME}.all_coding_NS.txt \
--recode vcf-fid bgz \
--out ${GENE_NAME}/chr${CHR_N}_${GENE_NAME}.all_coding_NS && \
tabix -f -p vcf ${GENE_NAME}/chr${CHR_N}_${GENE_NAME}.all_coding_NS.vcf.gz && \
~/bin/rvtests/executable/rvtest \
--noweb \
--inVcf ${GENE_NAME}/chr${CHR_N}_${GENE_NAME}.all_coding_NS.vcf.gz \
--kernel skat,skato \
--geneFile refFlat.txt \
--pheno ${GENE_NAME}/chr${CHR_N}_${GENE_NAME}_covariateFile.txt \
--pheno-name PHENO \
--covar ${GENE_NAME}/chr${CHR_N}_${GENE_NAME}_covariateFile.txt \
--covar-name SEX,AGE \
--out ${GENE_NAME}/chr${CHR_N}_${GENE_NAME}.all_coding_NS \
--numThread `nproc` \
--freqUpper 0.01 && \
awk '$9 ~ /nonsyn/ || $6 ~ /^splicing$/ || $9 ~ /stop/ || $9 ~ /frame/ {print $1" "$2" "$2" "$7}' \
${GENE_NAME}/chr${CHR_N}_${GENE_NAME}_recode_convert.annovar.trimmed.txt > ${GENE_NAME}/chr${CHR_N}_${GENE_NAME}.all_functional.txt && \
~/bin/plink \
--bfile ${GENE_NAME}/chr${CHR_N}_${GENE_NAME} \
--extract range ${GENE_NAME}/chr${CHR_N}_${GENE_NAME}.all_functional.txt \
--recode vcf-fid bgz \
--out ${GENE_NAME}/chr${CHR_N}_${GENE_NAME}.all_functional && \
tabix -f -p vcf ${GENE_NAME}/chr${CHR_N}_${GENE_NAME}.all_functional.vcf.gz && \
~/bin/rvtests/executable/rvtest \
--noweb \
--inVcf ${GENE_NAME}/chr${CHR_N}_${GENE_NAME}.all_functional.vcf.gz \
--kernel skat,skato \
--geneFile refFlat.txt \
--pheno ${GENE_NAME}/chr${CHR_N}_${GENE_NAME}_covariateFile.txt \
--pheno-name PHENO \
--covar ${GENE_NAME}/chr${CHR_N}_${GENE_NAME}_covariateFile.txt \
--covar-name SEX,AGE \
--out ${GENE_NAME}/chr${CHR_N}_${GENE_NAME}.all_functional \
--numThread `nproc` \
--freqUpper 0.01 && \
awk 'NR>1 && $31 > 12.37 {print $1" "$2" "$2" "$7}' \
${GENE_NAME}/chr${CHR_N}_${GENE_NAME}_recode_convert.annovar.trimmed.txt > ${GENE_NAME}/chr${CHR_N}_${GENE_NAME}.CADD.txt && \
~/bin/plink \
--bfile ${GENE_NAME}/chr${CHR_N}_${GENE_NAME} \
--extract range ${GENE_NAME}/chr${CHR_N}_${GENE_NAME}.CADD.txt \
--recode vcf-fid bgz \
--out ${GENE_NAME}/chr${CHR_N}_${GENE_NAME}.CADD && \
tabix -f -p vcf ${GENE_NAME}/chr${CHR_N}_${GENE_NAME}.CADD.vcf.gz && \
~/bin/rvtests/executable/rvtest \
--noweb \
--inVcf ${GENE_NAME}/chr${CHR_N}_${GENE_NAME}.CADD.vcf.gz \
--kernel skat,skato \
--geneFile refFlat.txt \
--pheno ${GENE_NAME}/chr${CHR_N}_${GENE_NAME}_covariateFile.txt \
--pheno-name PHENO \
--covar ${GENE_NAME}/chr${CHR_N}_${GENE_NAME}_covariateFile.txt \
--covar-name SEX,AGE \
--out ${GENE_NAME}/chr${CHR_N}_${GENE_NAME}.CADD \
--numThread `nproc` \
--freqUpper 0.01; \
done >list_genes.log 2>&1